# Time-series analsysis on River Meschio levels

### *in progress...*

### citation:
### data source: Agenzia Regionale per la Prevenzione e Protezione Ambientale (ARPA) del Veneto (www.arpa.veneto.it)

<figure>
    <img src="./imgs/i1.jpeg" alt="Meschio">
        <figcaption>River Meschio</figcaption>
</figure> 


In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as rq
import numpy as np
# custom script to load data from ARPAV site
import load_data
import pickle
# data Viz
from bokeh.plotting import *
from bokeh.models import *

In [6]:
DOWNLOAD_DATA = False # True: download data from ARPAV site, False: load from pickle file

In [7]:
if DOWNLOAD_DATA:
    years_list = np.arange(2010, 2021, 1).astype('str')
    stations_list = ['0463', '0462', '0296']
    wide_df = load_data.levels_to_df(years_list, stations_list, True)
else:
    with open('wide_df.pkl', 'rb') as f:
        wide_df = pickle.load(f)

In [8]:
# restructure df in long format
long_df = wide_df.melt(id_vars=['year','station','lev_type','day'], var_name='month', value_name='level').dropna()
long_df = long_df[long_df.level != -999.99]
long_df['date'] = long_df.apply(lambda x : pd.to_datetime( str(x['year']) +'-' + x['month'] +'-' + str(x['day']), format='%Y-%b-%d'), axis=1)
long_df = long_df.drop(["day","month","year"], axis=1)
long_df.set_index('date',inplace=True)

In [9]:
# create one data frame for each station
st_0463 = long_df.query('station=="0463"')[['lev_type','level']].pivot(columns='lev_type', values='level').sort_index()
st_0462 = long_df.query('station=="0462"')[['lev_type','level']].pivot(columns='lev_type', values='level').sort_index()
st_0296 = long_df.query('station=="0296"')[['lev_type','level']].pivot(columns='lev_type', values='level').sort_index()

In [10]:
# plot time-series
output_notebook()

h, w, ylim = 500, 500, (0.0, 3.0)

s_0 = ColumnDataSource(data=st_0463)

p_0 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim, title="Min levels")
p_0.line(x='date',y='MIN',  source=s_0)
p_0.xaxis.ticker=YearsTicker(desired_num_ticks=11)
p_0.yaxis.axis_label = 'Vittorio V.to'

p_1 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim, title="Average levels")
p_1.line(x='date',y='AVG',  source=s_0)
p_1.xaxis.ticker=YearsTicker(desired_num_ticks=11)

p_2 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim, title="Max levels")
p_2.line(x='date',y='MAX',  source=s_0)
p_2.xaxis.ticker=YearsTicker(desired_num_ticks=11)

s_1 = ColumnDataSource(data=st_0462)

p_3 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_3.line(x='date',y='MIN',  source=s_1)
p_3.xaxis.ticker=YearsTicker(desired_num_ticks=11)
p_3.yaxis.axis_label = 'Borgo Campion'

p_4 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_4.line(x='date',y='AVG',  source=s_1)
p_4.xaxis.ticker=YearsTicker(desired_num_ticks=11)

p_5 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_5.line(x='date',y='MAX',  source=s_1)
p_5.xaxis.ticker=YearsTicker(desired_num_ticks=11)

s_2 = ColumnDataSource(data=st_0296)

p_6 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_6.line(x='date',y='MIN',  source=s_2)
p_6.xaxis.ticker=YearsTicker(desired_num_ticks=11)
p_6.yaxis.axis_label = 'Cordignano'

p_7 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_7.line(x='date',y='AVG',  source=s_2)
p_7.xaxis.ticker=YearsTicker(desired_num_ticks=11)

p_8 = figure(height=h, width=w, x_axis_type='datetime',y_range=ylim)
p_8.line(x='date',y='MAX',  source=s_2)
p_8.xaxis.ticker=YearsTicker(desired_num_ticks=11)


grid=gridplot([[p_0, p_1, p_2],
               [p_3, p_4, p_5],
               [p_6, p_7, p_8]])
show(grid)

Loading BokehJS ...